In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends

import pickle
import pyExSi

import LadiskDAQ
import os

%matplotlib qt

In [2]:

acq1 = LadiskDAQ.NIAcquisition("TestInputTask", acquisition_name="NI")


acq2 = LadiskDAQ.SerialAcquisition(port="COM3", baudrate=250000, timeout=5,
                                  byte_sequence=(("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb",
                                  end_bytes=b"\n",
                                  pretest_time= None,
                                  sample_rate=500.,
                                  acquisition_name="Arduino"
                                  )

acq3 = LadiskDAQ.WaveFormsAcquisition(
    channels=[0, 1],    sample_rate=10000, 
    channel_names=["Channel 0", "Channel 1"], acquisition_name="AD", device_number=None
)
acq3.configure_channels(
    input_range={0: (-1, 1), 1: (-1, 1)},
)

In [3]:
acq3.acquisition_name

'AD'

In [4]:
#create signal:
fs = 10000 # output sample rate
mp = 10

time_array = np.arange(fs * mp) / fs  

N = len(time_array)    # number of data points of time signal
t = np.arange(0,N)/fs  # time vector
M = N // 2 + 1         # number of data points in frequency vector
freq = np.arange(0, M, 1) * fs / N # frequency vector
PSD = pyExSi.get_psd(freq, freq_lower=100, freq_upper=500) # one-sided flat-shaped PSD
signal1 = pyExSi.random_gaussian(N, PSD, fs)

signal2 = np.sin(2*np.pi*300*time_array)

excitation_signal = np.array([signal1, signal2])
plt.figure()
plt.plot(excitation_signal.T)

# create generation class:
gen1 = LadiskDAQ.NIGenerator('TestOutputTask', excitation_signal)

In [17]:
def multiply(self, channel_data):
    '''
    :param self:         instance of the acquisition object (has to be there so the function is called properly)
    :param channel_data: channel data
    '''
    return channel_data * 0.01
        
subplot_options = {
            (0, 0): {
                'xlim': (0, 0.1),
                'ylim': (-3, 3),
                'axis_style': 'linear'
            },
            (0, 1): {
                'xlim': (0, 0.1),
                'ylim': (-2, 2),
                'axis_style': 'linear'
            },
            (1, 0): {
                'xlim': (0, 600),
                'axis_style': 'log'
            },
            (1, 1): {
                'xlim': (0, 0.3),
                'axis_style': 'linear'
            }
        }
layout = {
    "NI": {
        (0, 0): [0],
        (0, 1): [1],
        (1, 0): [0, "fft"]
    },
    "AD": {
        (0, 0): [0],
        (0, 1): [1],
        (1, 0): [0, "fft"]
    },
    "Arduino": {
        (1, 1): [0, 1, multiply]
    }
}

vis = LadiskDAQ.Visualization(layout=layout, subplot_options=subplot_options, 
                              nth=10, refresh_rate=100)
#vis.max_points_to_refresh = 100000

In [18]:
# create DAQ object:
ldaq = LadiskDAQ.Core(acquisitions=[acq1, acq2, acq3], generations=[gen1], visualization=vis)

ldaq.set_trigger(
    source="NI",
    level    =1e10, 
    channel  =1,
    duration =10, 
    duration_unit='seconds', 
    presamples = 0, 
    trigger_type ='abs'
)

# ldaq.set_global_trigger(
#     "NI_task", 0, 1.4
# )

In [19]:
ldaq.run(measurement_duration=10., autostart=False, run_name="RunTest", save_interval=3.0, root='C:/LADISK/packages/test_meas', verbose=1)

	Waiting for trigger...saved.
saved.
Measurement finished.


In [14]:
measurement_dict = ldaq.get_measurement_dict(N_seconds=None)

root = r"C:\LADISK\packages\test_meas"
file = os.listdir(root)[-1]
measurement_dict_saved = LadiskDAQ.load_measurement(file, root)

In [15]:
fig, axs = plt.subplots(4, sharex=True)
for idx, source in enumerate(list(measurement_dict.keys())):
    meas = measurement_dict[source]
   
    meas_saved = measurement_dict_saved[source]
    axs[idx].plot( meas["time"], meas["data"], label="measurement")
    axs[idx].plot( meas_saved["time"], meas_saved["data"], label="saved")
    
    #axs[idx+1].plot( meas["time"], meas["data"], label="measurement")
    #axs[idx+2].plot( meas_saved["time"], meas_saved["data"], label="saved")
    
    #axs[idx+2].plot( meas["data"])
    #axs[idx+3].plot( meas_saved["data"])
    
for ax in axs:
    ax.legend(loc="upper center")


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [9]:
acq1.tshift, acq3.tshift, acq1.tshift - acq3.tshift

(0.04137538909912109, 0.036010456085205075, 0.0053649330139160165)

In [8]:
acq1.get_data("new")

(array([], dtype=float64), array([], shape=(0, 2), dtype=float64))

In [14]:
data = measurement_dict['NI_task']['data']
data_saved = measurement_dict_saved['NI_task']['data']
data.shape, data_saved.shape, acq1.Trigger.ringbuff.rows

((256000, 2), (256000, 2), 256000)

In [15]:
i = np.where(data[:, 1] > 1.5)[0][0]
i_saved = np.where(data_saved[:, 1] > 1.5)[0][0]
i, i_saved, i_saved-i

(0, 0, 0)

In [3]:
20 / (0.18 * (1000/750) / 750)

62500.00000000001